In [1]:
from rdkit import Chem
from rdkit.Chem import AllChem

def fragment_molecule(smiles):
    # Load the molecule
    mol = Chem.MolFromSmiles(smiles)
    if mol is None:
        raise ValueError(f"Invalid SMILES string: {smiles}")
    
    # Find all bonds in the molecule
    bonds = [(bond.GetBeginAtomIdx(), bond.GetEndAtomIdx()) for bond in mol.GetBonds()]
    
    # Store results
    fragments = []

    # Iterate over each bond
    for bond in bonds:
        a1, a2 = bond
        
        # Check if bond is part of a ring
        if mol.GetBondBetweenAtoms(a1, a2).IsInRing():
            # Find another bond in the ring to break
            ring_bonds = [b.GetIdx() for b in mol.GetBondBetweenAtoms(a1, a2).GetOwningMol().GetRingInfo().BondRings()]
            if len(ring_bonds) < 2:
                continue  # Skip if we cannot find two bonds in a ring
            
            # Fragment on two bonds to break the ring
            frag_mol = Chem.FragmentOnBonds(mol, [ring_bonds[0], ring_bonds[1]], addDummies=True, dummyLabels=[(0, 0)])
        else:
            # Fragment on a single bond
            bond_idx = mol.GetBondBetweenAtoms(a1, a2).GetIdx()
            frag_mol = Chem.FragmentOnBonds(mol, [bond_idx], addDummies=True, dummyLabels=[(0, 0)])
        
        # Split the fragmented molecule into individual fragments
        frags = Chem.GetMolFrags(frag_mol, asMols=True)
        
        # Convert fragments to SMILES
        frag_smiles = [Chem.MolToSmiles(frag, isomericSmiles=True) for frag in frags]
        
        # Only add pairs of fragments (skip if more than two fragments are generated)
        if len(frag_smiles) == 2:
            fragments.append(frag_smiles)
    
    return fragments

# Example usage
smiles = 'CCOC(c5nn(Cc6cnc(N7CCN(CC7)C8CC8)cc6)c(C)c5)=O'
result = fragment_molecule(smiles)
for frag_pair in result:
    print(f"Fragment pair: {frag_pair}")

AttributeError: 'tuple' object has no attribute 'GetIdx'